In [0]:
#from google.colab import drive
#drive.mount("/content/gdrive")

In [2]:
import numpy as np
import gensim
import string

from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation, TimeDistributed
from keras.models import Sequential
from keras.utils.data_utils import get_file

Using TensorFlow backend.


In [3]:
print('\nFetching the text...')
contentFile = "/content/gdrive/My Drive/Coding/datadump/combinedDump.txt"

print('\nPreparing the messages...')
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    return content

combinedContent = read_data(contentFile)
trainContent = combinedContent[:(9*len(combinedContent))//10]
testContent = combinedContent[(9*len(combinedContent))//10:]

max_message_len = 10

def readInputContent(fileContent):
  for i in range(len(fileContent)//max_message_len):
    yield fileContent[i*max_message_len:(i+1)*max_message_len]

#def readInputTest(fileContent):
#  for i in range(len(fileContent)//max_message_len - 1):
#    yield fileContent[i*max_message_len+(max_message_len//2):(i+1)*max_message_len+(max_message_len//2)]

def readInputWord2vec(inputFile):
    with open(inputFile, 'r') as f:
        for line in f:
              yield line.strip().split()

messages = list(readInputContent(trainContent))
testData = list(readInputContent(testContent))

word2vecData = list(readInputWord2vec(contentFile))

#messages += testData

print('Num messages:', len(messages))


Fetching the text...

Preparing the messages...
Num messages: 28511


In [0]:
class EpochLogger(gensim.models.callbacks.CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0

  def on_epoch_begin(self, model):
    pass

  def on_epoch_end(self, model):
    if self.epoch % 5 == 0:
      print("Epoch #{} end".format(self.epoch))
    self.epoch += 1

In [5]:
print('\nTraining word2vec...')
epochLogger = EpochLogger()
word_model = gensim.models.Word2Vec(word2vecData, size=128, min_count=1, window=3, compute_loss=True, iter=30, callbacks=[epochLogger])
print("Loss:", word_model.get_latest_training_loss())
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
print('Checking similar words:')
for word in ['fra', 'på', 'ok', 'nice', 'chunnoo', '[period]']:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.wv.most_similar(word)[:8])
  print('  %s -> %s' % (word, most_similar))


Training word2vec...
Epoch #0 end
Epoch #5 end
Epoch #10 end
Epoch #15 end
Epoch #20 end
Epoch #25 end
Loss: 4320308.0
Result embedding shape: (15040, 128)
Checking similar words:
  fra -> ifra (0.45), frem (0.43), igjen (0.40), ved (0.39), hjæm (0.38), sende (0.38), fram (0.37), grilltralla (0.37)
  på -> hjem (0.43), ved (0.36), mot (0.35), læk (0.33), fortsatt (0.33), forresten (0.32), over (0.32), film (0.32)
  ok -> okei (0.68), aha (0.68), oja (0.63), oki (0.59), sant (0.59), ah (0.59), je (0.58), spiritismen (0.58)
  nice -> nais (0.83), sweet (0.76), sårry (0.71), smud (0.69), fett (0.68), nicw (0.68), deilig (0.68), perfekt (0.67)
  chunnoo -> martinrygg (0.67), chun (0.63), niko (0.58), sig (0.58), mememagic (0.57), hadwul (0.56), sigmund (0.55), martin (0.54)
  [period] -> errors (0.49), adb (0.47), beint (0.44), skriftlig (0.44), klatring (0.44), [exclamationmark] (0.43), isj (0.43), typ (0.42)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [6]:
def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

print('\nPreparing the data for LSTM...')
"""
train_x = np.zeros([len(messages), max_message_len], dtype=np.int32)
train_y = np.zeros([len(messages)], dtype=np.int32)
for i, sentence in enumerate(messages):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
  
test_x = np.zeros([len(testData), max_message_len], dtype=np.int32)
test_y = np.zeros([len(testData)], dtype=np.int32)
for i, sentence in enumerate(testData):
  for t, word in enumerate(sentence[:-1]):
    test_x[i, t] = word2idx(word)
  test_y[i] = word2idx(sentence[-1])
"""  

train_x = np.zeros([len(messages), max_message_len], dtype=np.int32)
train_y = np.zeros([len(messages), max_message_len, 1], dtype=np.int32)
for i, sentence in enumerate(messages):
  for j in range(len(sentence)-1):
    train_x[i, j] = word2idx(sentence[j])
    train_y[i, j, 0] = word2idx(sentence[j+1])
    
test_x = np.zeros([len(testData), max_message_len], dtype=np.int32)
test_y = np.zeros([len(testData), max_message_len, 1], dtype=np.int32)
for i, sentence in enumerate(testData):
  for j in range(len(sentence)-1):
    test_x[i, j] = word2idx(sentence[j])
    test_y[i, j, 0] = word2idx(sentence[j+1])

###Attempt at generating full dentences from full sentences
#train_x = np.zeros([len(word2vecData), max_message_len], dtype=np.int32)
#train_y = np.zeros([len(word2vecData), max_message_len, 1], dtype=np.int32)
#for i in range(len(word2vecData)-1):
#  for j in range(max_message_len):
#    train_x[i, j] = word2idx(word2vecData[i][j]) if j < len(word2vecData[i]) else 0
#    train_y[i, j, 0] = word2idx(word2vecData[i+1][j]) if j < len(word2vecData[i+1]) else 0

###Don't know
#def read_data(fname):
#    with open(fname) as f:
#        content = f.readlines()
#    content = [x.strip() for x in content]
#    content = [word for i in range(len(content)) for word in content[i].split()[:max_message_len]]
#    content = np.array(content)
#    return content
  
#trainData = read_data(path)

#inputLength = 10
#batchLength = len(trainData)//(inputLength + 1)
#train_x = np.zeros([batchLength, inputLength], dtype=np.int32)
#train_y = np.zeros([batchLength], dtype=np.int32)
#for i in range(batchLength):
#  for j in range(inputLength):
#    train_x[i, j] = word2idx(trainData[i*(inputLength+1) + j])
#  train_y[i] = word2idx(trainData[(i+1)*(inputLength+1)-1])


print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)
print('test_x shape:', test_x.shape)
print('test_y shape:', test_y.shape)


Preparing the data for LSTM...
train_x shape: (28511, 10)
train_y shape: (28511, 10, 1)
test_x shape: (3168, 10)
test_y shape: (3168, 10, 1)


In [76]:
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
#model.add(LSTM(units=emdedding_size, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
#model.add(LSTM(units=emdedding_size, return_sequences=True, recurrent_dropout=0.5))
model.add(LSTM(units=emdedding_size, return_sequences=True, dropout=0.25, recurrent_dropout=0.25))
model.add(TimeDistributed(Dense(units=vocab_size)))
model.add(Activation('softmax'))
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
print(model.summary())


Training LSTM...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 128)         1925120   
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 15040)       1940160   
_________________________________________________________________
activation_4 (Activation)    (None, None, 15040)       0         
Total params: 3,996,864
Trainable params: 3,996,864
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds), preds[np.argmax(preds)]
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas), preds[np.argmax(probas)]

def generate_next(text, num_generated=30):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs))
    idx, _ = sample(prediction[-1][-1], temperature=0.7)
    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)

###Attepmt at generating full sentences from full sentences
"""
def newGenerateResult(text):
  inputIdxs = [word2idx(word) for word in text.lower().split()]
  for i in range(max_message_len - len(inputIdxs)):
    inputIdxs.append(0)
  prediction = model.predict(x=np.array(inputIdxs))
  outputIdxs = [np.argmax(preds) for preds in prediction]
  return ' '.join(idx2word(idx) for idx in outputIdxs)
"""

def on_epoch_end(epoch, _):
  if epoch % 1 == 0:
    print('\nGenerating text after epoch: %d' % epoch)
    texts = [
      'jeg har en [message]',
      'når går dokker [period] [message]',
      'æ kan [message]',
      'tja [message]',
    ]
    for text in texts:
      sample = generate_next(text)
      #sample = newGenerateResult(text)
      print('%s... -> %s' % (text, sample))
    print(model.evaluate(x=test_x, y=test_y, batch_size=32))

In [78]:
model.fit(train_x, train_y,
          batch_size=32,
          epochs=10,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])



Epoch 1/10
28511/28511 [==============================] - 20s 712us/step - loss: 5.4588

Generating text after epoch: 0
jeg har en [message]... -> jeg har en [message] æ dra en tur [message] hvor [message] [number] [comma] da [questionmark] [message] står ikke møtes [message] hmm [message] hadd jo det er det e det er ikke kan vær på
når går dokker [period] [message]... -> når går dokker [period] [message] du bruke en tur [dash] [misc] [rightbracket] [message] æ er med [message] får alt e det ligg en tur [message] haha [message] la d kl å si det e heller
æ kan [message]... -> æ kan [message] vi har desverre [message] humor [message] damn [message] mhm [message] den som får vi e d [message] går på løsninga [message] haha [message] e grei [period] ja [comma] så e
tja [message]... -> tja [message] kan kom å dra det [comma] men [message] haha [exclamationmark] [message] klatring [comma] så får vel [questionmark] [message] [message] ja [message] sjer ass [comma] det kjæm på sjø [questionmark

In [10]:
testStr = "ok [message] men e bare"
wordIdx = [word2idx(word) for word in testStr.split()]
prediction = model.predict(x=np.array(wordIdx))
#print(np.argmax(prediction[-1]))
#print(prediction)
for i in range(len(prediction)):
  print(idx2word(np.argmax(prediction[i])))

[message]
æ
det
det
å


In [0]:
def generateNext2(text, num_generated=30, temperature=0):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  prbs = [1.0 for word in text.lower().split()]
  for i in range(num_generated):
    inpt = np.zeros([1, len(word_idxs)], dtype=np.int32)
    for i, idx in enumerate(word_idxs):
      inpt[0, i] = idx
    prediction = model.predict(x=inpt)
    idx, prb = sample(prediction[-1][-1], temperature=temperature)
    word_idxs.append(idx)
    prbs.append(prb)
  return [idx2word(idx) for idx in word_idxs], prbs

In [123]:
gnr, prbs = generateNext2("takk [message]", temperature=0.6)
print(gnr)
print(list(zip(gnr, prbs)))
n = 0
s = 0
st = False
for w, p in list(zip(gnr, prbs)):
  if not st and w == "[message]":
    st = True
  elif st:
    n += 1
    s += p
    if w == "[message]":
      break
      
print(s/n)

['takk', '[message]', '[misc]', '[message]', 'e', 'd', 'bare', 'å', 'gjøre', 'væl', 'kveld', '[period]', 'mikael', '[questionmark]', '[questionmark]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]', '[message]']
[('takk', 1.0), ('[message]', 1.0), ('[misc]', 0.004255437679862575), ('[message]', 0.701659277775599), ('e', 0.02189985580094615), ('d', 0.41992602449618166), ('bare', 0.09062807266820994), ('å', 0.7926176763659606), ('gjøre', 0.009243483083222377), ('væl', 0.0029328335074224423), ('kveld', 1.657804492832282e-05), ('[period]', 0.04565264534526417), ('mikael', 0.0007189806183594381), ('[questionmark]', 0.4637474183186627), ('[questionmark]', 0.0020096383372261487), ('[message]', 0.980269752594759), ('[message]', 0.9997142076522545), ('[message]', 0.9999999809767677), ('[message]', 0.9999999233259925), ('[message]', 0.999

In [13]:
print(model.predict(x=np.array([word2idx(word) for word in "ok [message]".lower().split()])).shape)

(2, 1, 15040)


In [14]:
print(word_model.most_similar("ka"))

[('koss', 0.6037603616714478), ('korsn', 0.5622408390045166), ('kæm', 0.5329692959785461), ('tenerif', 0.5194030404090881), ('kor', 0.4969877600669861), ('nokka', 0.4946803152561188), ('nåkka', 0.47311800718307495), ('lærebøker', 0.442679226398468), ('kossn', 0.4411323070526123), ('ting', 0.43210580945014954)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
word_model.save("/content/gdrive/My Drive/Coding/models/word2vecModel.model")
model.save("/content/gdrive/My Drive/Coding/models/lstmModel.h5", overwrite=True)